In [1]:
import sys
import os

# Añadir la ruta a sys.path
sys.path.append(os.path.abspath('../lib/intro_ia/clase2/hanoi_tower'))

import aima
import hanoi_states
import search


1. PEAS del problema

| Agente | Performance | Environment | Actuators | Sensors |
|--------|-------------|-------------|-----------|---------|
|Torre de Hanoi | Utilizar el menor numero de movimientos posibles| Discos y torres  | Mover discos respetando las reglas del juego| Posición de los discos y estado de las torres|

2. Propiedades del Entorno:

a. Totalmente observable: Sabemos exactamente todas las posiciones de los discos y el estado de las torres.
b. Determinístico: Las acciones tienen resultados predecibles.
c. Secuencial: Cada movimiento de un disco afecta el estado del entorno y las opciones disponibles para los movimientos futuros.
d. Estático: No hay cambios de estados mientras que el agente no intervenga.
e. Discreto: Los estados y acciones son finitos y contables.
f. Agente individual: Solo existe un agente realizando acciones en el entorno.

3.
- Estado: Ubicación de los discos en las torres en un momento específico.
- Espacio de estados: Cantidad de posiciones posibles para los discos en las torres.
- Árbol de búsqueda:
- Nodo de búsqueda:
- Objetivo : Configuración en la que todo los discos han sido movidos de la torre inicial a la torre objetivo.
- Acción: Mover un disco de una torre a otra, siempre y cuando éste no se apoye sobre un disco más chico.
- Frontera: Aquella combinación de posiciones que ya fueron tomadas por el agente versus las posiciones que todavia no fueron exploradas.